<a href="https://colab.research.google.com/github/olivierguntern/LIGP/blob/main/P8_01_script_02_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Segmentation des images unet

### Importer les librairies necessaires

In [2]:
pip install imgaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image, display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.metrics import MeanIoU

import random

import imgaug as ia
from imgaug import augmenters as iaa

from sklearn.model_selection import train_test_split

import warnings

### Identification des différents dossiers d'images

In [4]:
leftImg8bit_path = '/content/drive/MyDrive/data/leftImg8bit'
gtFine_path = '/content/drive/MyDrive/data/gtFine'

In [5]:
# Affiche la structure et le nombre de fichiers du dossier
def printStructure(dir_path):
    list_dir = (os.listdir(dir_path)) # trier les noms de dossiers dans l'ordre alphabétique
    list_dir_country = {}
    subdir_list = [] # initialiser une liste vide pour chaque répertoire

    nbr_data = {}
    nbr_dir = len(list_dir) # obtenir le nombre de répertoires dans le dossier

    for dir in list_dir:
        list_dir_country[dir] = sorted(os.listdir(os.path.join(dir_path, dir)))
        
    print(dir_path)
    for i in list_dir:
        print(f"    {i}/")
        nbr_data[i] = 0
        nbr_subdir = len(list_dir_country[i]) # obtenir le nombre de sous-répertoires dans chaque répertoire
        for j in list_dir_country[i]:
            nbr_data_country = len(os.listdir(os.path.join(dir_path, i, j)))
            nbr_data[i] += nbr_data_country
            print (f"       {j}/ {nbr_data_country}")
            subdir_list.append(j) # ajouter le nom de chaque sous-répertoire à la liste correspondante
    print(f"\nTotal : {nbr_dir} répertoires")
    print(f"\nTotal : {nbr_subdir} sous répertoires")
    for i in list_dir:
        print(f"    {i}: {nbr_data[i]}")
    return subdir_list

In [6]:
train_villes=printStructure(leftImg8bit_path)

/content/drive/MyDrive/data/leftImg8bit
    train/
       aachen/ 174
       bochum/ 96
       bremen/ 316
       cologne/ 158
       darmstadt/ 85
       dusseldorf/ 221
       erfurt/ 109
       hamburg/ 248
       hanover/ 196
       jena/ 119
       krefeld/ 99
       monchengladbach/ 94
       strasbourg/ 365
       stuttgart/ 196
       tubingen/ 144
       ulm/ 95
       weimar/ 142
       zurich/ 122

Total : 1 répertoires

Total : 18 sous répertoires
    train: 2979


In [7]:
train_villes

['aachen',
 'bochum',
 'bremen',
 'cologne',
 'darmstadt',
 'dusseldorf',
 'erfurt',
 'hamburg',
 'hanover',
 'jena',
 'krefeld',
 'monchengladbach',
 'strasbourg',
 'stuttgart',
 'tubingen',
 'ulm',
 'weimar',
 'zurich']

In [8]:
mask_villes=printStructure(gtFine_path)

/content/drive/MyDrive/data/gtFine
    train/
       aachen/ 696
       bochum/ 384
       bremen/ 1264
       cologne/ 616
       darmstadt/ 340
       dusseldorf/ 884
       erfurt/ 436
       hamburg/ 992
       hanover/ 784
       jena/ 476
       krefeld/ 396
       monchengladbach/ 376
       strasbourg/ 1460
       stuttgart/ 784
       tubingen/ 576
       ulm/ 380
       weimar/ 568
       zurich/ 488

Total : 1 répertoires

Total : 18 sous répertoires
    train: 11900


In [9]:
train_image_paths = []
train_mask_paths = []

for villes in train_villes:


   train_image_paths.append(f"{leftImg8bit_path}/train/{villes}/")
   train_image_paths.append(f"{gtFine_path}/train/{villes}/")

# Créer des dictionnaires vides pour les listes de chemins de chaque ville et type d'image
leftImg8bit_paths = {}
gtFine_paths = {}

# Parcourir les listes de chemins pour chaque image
for img_path in train_image_paths:
    # Extraire le nom de la ville à partir du chemin
    ville = img_path.split('/')[-2]
    
    # Vérifier si la ville est déjà présente dans les dictionnaires
    if ville not in leftImg8bit_paths:
        leftImg8bit_paths[ville] = []
        gtFine_paths[ville] = []
    
    # Ajouter le chemin à la liste correspondante en fonction du type d'image
    if 'leftImg8bit' in img_path:
        leftImg8bit_paths[ville].append(img_path)
    elif 'gtFine' in img_path:
        gtFine_paths[ville].append(img_path)

    


In [10]:
train_image_paths

['/content/drive/MyDrive/data/leftImg8bit/train/aachen/',
 '/content/drive/MyDrive/data/gtFine/train/aachen/',
 '/content/drive/MyDrive/data/leftImg8bit/train/bochum/',
 '/content/drive/MyDrive/data/gtFine/train/bochum/',
 '/content/drive/MyDrive/data/leftImg8bit/train/bremen/',
 '/content/drive/MyDrive/data/gtFine/train/bremen/',
 '/content/drive/MyDrive/data/leftImg8bit/train/cologne/',
 '/content/drive/MyDrive/data/gtFine/train/cologne/',
 '/content/drive/MyDrive/data/leftImg8bit/train/darmstadt/',
 '/content/drive/MyDrive/data/gtFine/train/darmstadt/',
 '/content/drive/MyDrive/data/leftImg8bit/train/dusseldorf/',
 '/content/drive/MyDrive/data/gtFine/train/dusseldorf/',
 '/content/drive/MyDrive/data/leftImg8bit/train/erfurt/',
 '/content/drive/MyDrive/data/gtFine/train/erfurt/',
 '/content/drive/MyDrive/data/leftImg8bit/train/hamburg/',
 '/content/drive/MyDrive/data/gtFine/train/hamburg/',
 '/content/drive/MyDrive/data/leftImg8bit/train/hanover/',
 '/content/drive/MyDrive/data/gtFin

In [11]:
import os

# Dictionnaire pour stocker les chemins complets des images, avec les villes comme clé
all_image_paths = {}

for path in train_image_paths:
    ville = path.split("train/")[1].split("/")[0]
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".png"):
                all_image_paths.setdefault(ville, []).append(os.path.join(root, file))



In [12]:
import os

# Chemin des images
image_path = '/content/drive/MyDrive/data/leftImg8bit/train/'

# Clé
key = 'zurich'

# Récupération des chemins complets des images pour la clé donnée
image_paths = [os.path.join(root, file) for root, _, files in os.walk(image_path + key) for file in files if file.endswith('.png')]

# Affichage des 5 premières images
for i in range(5):
    print(image_paths[i])


/content/drive/MyDrive/data/leftImg8bit/train/zurich/zurich_000074_000019_leftImg8bit.png
/content/drive/MyDrive/data/leftImg8bit/train/zurich/zurich_000103_000019_leftImg8bit.png
/content/drive/MyDrive/data/leftImg8bit/train/zurich/zurich_000040_000019_leftImg8bit.png
/content/drive/MyDrive/data/leftImg8bit/train/zurich/zurich_000019_000019_leftImg8bit.png
/content/drive/MyDrive/data/leftImg8bit/train/zurich/zurich_000037_000019_leftImg8bit.png


### Dèfinir différente paramètres

In [13]:
img_size = (128 , 128)
num_classes = 8
batch_size = 2975
imgaug_multiplier = 2
epochs = 10
patience = 5
AUTO = tf.data.AUTOTUNE

### Dèfinir les fonctions me permettant de passer de 32 sous catégories à seulement 8 catégories

In [14]:
categories = {
    'void': [0, 1, 2, 3, 4, 5, 6],
    'flat': [7, 8, 9, 10],
    'construction': [11, 12, 13, 14, 15, 16],
    'object': [17, 18, 19, 20],
    'nature': [21, 22],
    'sky': [23],
    'human': [24, 25],
    'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]
}

def preprocessImg(img):
    categories = {elem: i for i, cat in enumerate(categories) for elem in cat}
    return np.vectorize(lambda x: categories.get(x, -1))(img)

### Dèfinir la fonction me permettant l'augmentation du nombre d'images

In [15]:
def generateRandomParams(seed):
    np.random.seed(seed)
    angle = np.random.randint(26)
    np.random.seed(seed*2)
    positive = np.random.randint(2)
    sigma = np.random.uniform(0, 1)
    
    if positive == 0:
        angle = angle * -1
        
    crop = np.random.randint(3)
    crop = crop / 10
    
    #print(angle, crop)
        
    return angle, crop, sigma

### Réaliser le gènèrateur de données

In [16]:
class Image(Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        
        x = np.zeros((self.batch_size * imgaug_multiplier,) + self.img_size + (3,), dtype="uint8")        
        for j, path in enumerate(batch_input_img_paths):
            img = image.load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size * imgaug_multiplier,) + self.img_size+ (1,), dtype="uint8")

        for j, path in enumerate(batch_target_img_paths):
            _img = image.load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = preprocessImg(_img)
            
            # Image AUGMENTATION         
        for mul in range(1, imgaug_multiplier):  
            for i in range(0, self.batch_size):
                
                
                angle, crop, sigma = generateRandomParams((1 + i) * mul)
                
                photo_aug = iaa.Sequential([                        
                    iaa.Affine(rotate=(angle)),
                    iaa.Crop(percent=(crop)),
                    iaa.GaussianBlur(sigma=(0.0, sigma))
                ])
                
                label_aug = iaa.Sequential([                        
                    iaa.Affine(rotate=(angle)),
                    iaa.Crop(percent=(crop)),
                ])
                

                image_aug = photo_aug(image=x[i])
                x[batch_size * mul + i] = image_aug

                image_aug = label_aug(image=y[i])
                y[batch_size * mul + i] = image_aug # END Image AUGMENTATION  
                
        return x, y

https://keras.io/examples/vision/oxford_pets_image_segmentation/

### Gènérer les images

In [17]:
train_seq = Image(
    batch_size, img_size, train_img_paths, train_ann_paths
)

NameError: ignored

### Vérifier la correspondance des images avec leurs masques respectifs

In [ ]:
assert train_seq[0][0].shape == (batch_size * imgaug_multiplier, *img_size, 3)
assert train_seq[0][1].shape == (batch_size * imgaug_multiplier, *img_size,1)

### Extraire les variables images et masques puis vérifier la forme

In [ ]:
for x, y in train_seq:
    break
x.shape, y.shape

### Contrôler le nombre maximal de pixel contenue dans la variable image

In [ ]:
print("Valeurs max pixels image: ", x.max())

### Contrôler le nombre de catègories prèsents dans la variable masque

In [ ]:
print("Valeurs pixels mask: ", np.unique(y))

### Normaliser les images

In [ ]:
image_dataset = x/255.  

### Fractionner mes images et mes masques en jeu d'entrainement et de test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_dataset, y, test_size = 0.2, random_state = 42)

### Etablir les 8 canaux de sortie avec conversion catègoriel de notre y_train et y_test(afin d'obtenir le bon format d'entrèe)

In [ ]:
train_masks_cat = to_categorical(y_train, num_classes=num_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], num_classes))

test_masks_cat = to_categorical(y_test, num_classes=num_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], num_classes))

print("Format entrée label entrainement:",y_train_cat.shape)
print("Format entrée label test:",y_test_cat.shape)

### Construire Unet en divisant l’encodeur et le décodeur en blocs
https://github.com/nikhilroxtomar/Unet-for-Person-Segmentation/


In [ ]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

### Bloc encodeur: conv_block suivi de maxpooling

In [ ]:
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p 

### Bloc décodeur (Les fonctions de saut obtiennent l’entrée de l’encodeur pour la concaténation)

In [ ]:
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

### Construire Unet en utilisant les blocs

In [ ]:
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1: 
        activation = 'sigmoid'
    else:
        activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

### Dèfinir la variable d'entrèe input_shape de mon modèle 

In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

### Construire mon modèle

In [ ]:
model = build_unet(input_shape, n_classes=8)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Entaîner mon modèle

In [ ]:
history = model.fit(X_train, y_train_cat, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=60, 
                    validation_data=(X_test, y_test_cat), 
                    shuffle=False)

### Enregistrer le modèle pour une utilisation ultérieure

In [ ]:
model.save('model_avance.h5')

### Tracer la précision et la perte de formation et de validation à chaque époque

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Perte de formation')
plt.plot(epochs, val_loss, 'r', label='Perte de validation')
plt.title('Perte de formation et de validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Précision formation')
plt.plot(epochs, val_acc, 'r', label='Précision validation')
plt.title('Précision formation et validation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### Charger le modèle précédemment enregistré

In [ ]:
model = load_model('model_avance.h5', compile=False)

### Prédiction sur les données test

In [ ]:
y_pred=model.predict(X_test)
y_pred.shape

In [ ]:
y_pred_argmax=np.argmax(y_pred, axis=3)
y_pred_argmax.shape

Celà me permet d'avoir une forme pour calculer l'IOU

### Utilisation de la fonction keras intégrée

In [ ]:
n_classes = 8
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Moyenne IoU =", IOU_keras.result().numpy())

En utilisant la mètrique de keras MeanIoU instancie la moyenne IOU puis met à jours l'état des résultats de la prédiction des masques que j'ai obtenu sur les données test en le comparant au label de l'image  

### Pour calculer I0U pour chaque classe (affichage de la matrice)

Type de matrice de confusion 

In [ ]:
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)

In [ ]:
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[0,4] + values[0,5] 
                          + values[0,6] + values[0,7] + values[1,0]+ values[2,0]+ values[3,0]+ values[4,0] 
                          + values[5,0] + values[6,0] + values[7,0])

class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[1,4] + values[1,5] 
                          + values[1,6] + values[1,7] + values[0,1]+ values[2,1]+ values[3,1]+ values[4,1] 
                          + values[5,1] + values[6,1] + values[7,1])

class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[2,4] + values[2,5] 
                          + values[2,6] + values[2,7] + values[0,2]+ values[1,2]+ values[3,2]+ values[4,2] 
                          + values[5,2] + values[6,2] + values[7,2])

class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[3,4] + values[3,5] 
                          + values[3,6] + values[3,7] + values[0,3]+ values[1,3]+ values[2,3]+ values[4,3] 
                          + values[5,3] + values[6,3] + values[7,3])

class5_IoU = values[4,4]/(values[4,4] + values[4,0] + values[4,1] + values[4,2] + values[4,3] + values[4,5] 
                          + values[4,6] + values[4,7] + values[0,4]+ values[1,4]+ values[2,4]+ values[3,4] 
                          + values[5,4] + values[6,4] + values[7,4])

class6_IoU = values[5,5]/(values[5,5] + values[5,0] + values[5,1] + values[5,2] + values[5,3] + values[5,4] 
                          + values[5,6] + values[5,7] + values[0,5]+ values[1,5]+ values[2,5]+ values[3,5] 
                          + values[4,5] + values[6,5] + values[7,5])

class7_IoU = values[6,6]/(values[6,6] + values[6,0] + values[6,1] + values[6,2] + values[6,3] + values[6,4] 
                          + values[6,5] + values[6,7] + values[0,6]+ values[1,6]+ values[2,6]+ values[3,6] 
                          + values[4,6] + values[5,6] + values[7,6])

class8_IoU = values[7,7]/(values[7,7] + values[7,0] + values[7,1] + values[7,2] + values[7,3] + values[7,4] 
                          + values[7,5] + values[7,6] + values[0,7]+ values[1,7]+ values[2,7]+ values[3,7] 
                          + values[4,7] + values[5,7] + values[6,7])

print("IoU pour void est: ", class1_IoU)
print("IoU pour flat est: ", class2_IoU)
print("IoU pour construction est: ", class3_IoU)
print("IoU pour objet est: ", class4_IoU)
print("IoU pour nature est: ", class5_IoU)
print("IoU pour sky est: ", class6_IoU)
print("IoU pour human est: ", class7_IoU)
print("IoU pour vehicle est: ", class8_IoU)

### Prédire sur une image

In [ ]:
img = load_img('jk/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/test/berlin/berlin_000000_000019_leftImg8bit.png', target_size=(128, 128)) 
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
x= x/255.
prediction = model.predict(x)
predicted_img=np.argmax(prediction, axis=3)[0,:,:]

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Image')
plt.imshow(img)

plt.subplot(232)
plt.title('Prediction mask')
plt.imshow(predicted_img, cmap='jet')
plt.show()